# Pytorch Modelling GPU 

Models are from **https://github.com/qubvel/segmentation_models.pytorch** 
Some of the lossses are from **https://github.com/JunMa11/SegLoss**

The base codes are from here -> https://www.kaggle.com/vineeth1999/hubmap-pytorch-efficientunet-offline
Please upvote this notebook as well

## Things that should be updated

>- parameter settings
>- loss functions
>- other optimizers
>- pth file save(find best pth in a fold)
>- more models(change encoder)
>- working without the Internet
>- inference
>- etc

## Imports

In [ ]:
from sklearn.model_selection import GroupKFold
import torch
from torch import nn
import torchvision
import cv2
import os
import numpy as np
import pandas as pd
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau, CosineAnnealingWarmRestarts
from scipy.ndimage.interpolation import zoom
import albumentations as A
from torch.nn import functional as F
from albumentations.pytorch import ToTensorV2

import matplotlib.pyplot as plt
import sys
import time
import random

In [ ]:
class CFG:
    data = 512 #256
    debug=False
    apex=False
    print_freq=100
    num_workers=4
    img_size=224 # appropriate input size for encoder 
    scheduler='CosineAnnealingWarmRestarts' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epoch=5 # Change epochs
    criterion= 'Lovasz' #'DiceBCELoss' # ['DiceLoss', 'Hausdorff', 'Lovasz']
    base_model='FPN' # ['Unet']
    encoder = 'se_resnet50' # ['efficientnet-b5'] or other encoders from smp
    lr=1e-4
    min_lr=1e-6
    batch_size=4
    weight_decay=1e-6
    gradient_accumulation_steps=1
    seed=2021
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    train=True
    inference=False
    optimizer = 'Adam'
    T_0=10
    # N=5 
    # M=9
    T_max=10
    #factor=0.2
    #patience=4
    #eps=1e-6
    smoothing=1

In [ ]:
def seed_torch(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
seed_torch(seed=CFG.seed)

RandAugment - I tried to use it but got some errors when used with albumentations

In [ ]:
'''
sys.path.append('../input/randaug')
from RandAugment import RandAugment
'''

## Dataset

In [ ]:
base_transform = A.Compose([
        A.Resize(CFG.img_size, CFG.img_size, p=1.0),
        A.HorizontalFlip(),
        A.VerticalFlip(),
        A.RandomRotate90(),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=20, p=0.9, 
                         border_mode=cv2.BORDER_REFLECT),
        A.OneOf([
            A.OpticalDistortion(p=0.4),
            A.GridDistortion(p=.1),
            A.IAAPiecewiseAffine(p=0.4),
        ], p=0.3),
        A.OneOf([
            A.HueSaturationValue(10,15,10),
            A.CLAHE(clip_limit=3),
            A.RandomBrightnessContrast(),            
        ], p=0.5),
        ToTensorV2()
    ], p=1.0)

'''
rand_transform = A.Compose([
            RandAugment(CFG.N, CFG.M),
            A.Transpose(p=0.5),
            A.VerticalFlip(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
            A.Resize(CFG.img_size, CFG.img_size, p=1.0),
            A.Normalize(),
            ToTensorV2()
        ])
'''

strong_transform = A.Compose([
            A.Transpose(p=0.5),
            A.VerticalFlip(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03, p=0.5),
            A.OneOf([
                    A.RandomGamma(),
                    A.GaussNoise()           
                ], p=0.5),
            A.OneOf([
                    A.OpticalDistortion(p=0.4),
                    A.GridDistortion(p=0.2),
                    A.IAAPiecewiseAffine(p=0.4),
                ], p=0.5),
            A.OneOf([
                    A.HueSaturationValue(10,15,10),
                    A.CLAHE(clip_limit=4),
                    A.RandomBrightnessContrast(),            
                ], p=0.5),
    
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
            A.Resize(CFG.img_size, CFG.img_size, p=1.0),
            ToTensorV2()
        ])

weak_transform = A.Compose([
        A.Resize(CFG.img_size, CFG.img_size, p=1.0),
        A.HorizontalFlip(),
        A.VerticalFlip(),
        A.RandomRotate90(),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=15, p=0.9, 
                         border_mode=cv2.BORDER_REFLECT),
        ToTensorV2()
    ], p=1.0)

val_transform = A.Compose([
        A.Resize(CFG.img_size, CFG.img_size, p=1.0),
        ToTensorV2()
    ], p=1.0)

In [ ]:
mean = np.array([0.65459856,0.48386562,0.69428385])
std = np.array([0.15167958,0.23584107,0.13146145])

class HuBMAPDataset(Dataset):
    def __init__(self,df, train='train', augment='weak', transform=True):
        ids = df.id.values
        #kf = KFold(n_splits=nfolds,random_state=SEED,shuffle=True)
        #ids = set(ids[list(kf.split(ids))[fold][0 if train else 1]])
        if CFG.data==512:
            self.fnames = [fname for fname in os.listdir('../input/hubmap-512x512/train') if fname.split('_')[0] in ids]
        elif CFG.data==256:
            self.fnames = [fname for fname in os.listdir('../input/hubmap-256x256/train') if fname.split('_')[0] in ids]
        self.train = train
        self.augment=augment
        self.transform = transform
        
    def __len__(self):
        return len(self.fnames)
    
    def __getitem__(self, idx):
        fname = self.fnames[idx]
        if CFG.data==512:
            img = cv2.cvtColor(cv2.imread(os.path.join('../input/hubmap-512x512/train',fname)), cv2.COLOR_BGR2RGB)
            mask = cv2.imread(os.path.join('../input/hubmap-512x512/masks',fname),cv2.IMREAD_GRAYSCALE)
        elif CFG.data==256:
            img = cv2.cvtColor(cv2.imread(os.path.join('../input/hubmap-256x256/train',fname)), cv2.COLOR_BGR2RGB)
            mask = cv2.imread(os.path.join('../input/hubmap-256x256/masks',fname),cv2.IMREAD_GRAYSCALE)
        
        if self.train == 'train':
            if self.transform == True:
                if self.augment == 'base':
                    augmented = base_transform(image=img,mask=mask)
                    img,mask = augmented['image'],augmented['mask']
                elif self.augment == 'weak':
                    augmented = weak_transform(image=img,mask=mask)
                    img,mask = augmented['image'],augmented['mask']
                elif self.augment == 'strong':
                    augmented = strong_transform(image=img,mask=mask)
                    img,mask = augmented['image'],augmented['mask']
                    
        elif self.train == 'val':
            transformed = val_transform(image=img,mask=mask)
            img,mask = transformed['image'],transformed['mask']
            
        img = img.type('torch.FloatTensor')
        img = img/255
        mask = mask.type('torch.FloatTensor')

        return img, mask
          

In [ ]:
train_df = pd.read_csv('../input/hubmap-kidney-segmentation/train.csv')

In [ ]:
train_df.head()

Visualize augmented data(weak) and mask

In [ ]:
# weak augmentation
train_dataset = HuBMAPDataset(train_df, train='train', augment='weak', transform=True)
plt.figure(figsize=(15,10))
for i in range(6):
    image, mask = train_dataset[i]
    plt.subplot(3,4,2*i+1)
    plt.imshow(np.transpose((image), (1,2,0)))
    plt.subplot(3,4,2*i+2)
    plt.imshow(mask)

Visualize augmented data and mask

In [ ]:
train_dataset = HuBMAPDataset(train_df, train='train', augment='base', transform=True)
plt.figure(figsize=(15,10))
for i in range(6):
    image, mask = train_dataset[i]
    plt.subplot(3,4,2*i+1)
    plt.imshow(np.transpose((image), (1,2,0)))
    plt.subplot(3,4,2*i+2)
    plt.imshow(mask)

Visualize augmented data(strong) and mask

In [ ]:
train_dataset = HuBMAPDataset(train_df, train='train', augment='strong', transform=True)
plt.figure(figsize=(15,10))
for i in range(6):
    image, mask = train_dataset[i]
    plt.subplot(3,4,2*i+1)
    plt.imshow(np.transpose((image), (1,2,0)))
    plt.subplot(3,4,2*i+2)
    plt.imshow(mask)

## DataLoader

In [ ]:
if CFG.data==512:
    directory_list = os.listdir('../input/hubmap-512x512/train')
elif CFG.data==256:
    directory_list = os.listdir('../input/hubmap-256x256/train')
directory_list = [fnames.split('_')[0] for fnames in directory_list]
dir_df = pd.DataFrame(directory_list, columns=['id'])
dir_df

In [ ]:
def prepare_train_valid_dataloader(df, fold):
    train_ids = df[~df.Folds.isin(fold)]
    val_ids = df[df.Folds.isin(fold)]
    
    train_ds = HuBMAPDataset(train_ids, train='train', weak=False, transform=True)
    val_ds = HuBMAPDataset(val_ids, train='train', weak=False, transform=True)
    train_loader = DataLoader(train_ds, batch_size=CFG.batch_size, pin_memory=True, shuffle=True, num_workers=CFG.num_workers)
    val_loader = DataLoader(val_ds, batch_size=CFG.batch_size, pin_memory=True, shuffle=False, num_workers=CFG.num_workers)
    return train_loader, val_loader

## Model

In [ ]:
!pip install ../input/segmentationmodelspytorch/segmentation_models/timm-0.1.20-py3-none-any.whl
!pip install ../input/segmentationmodelspytorch/segmentation_models/segmentation_models_pytorch-0.1.2-py3-none-any.whl

In [ ]:
sys.path.append('../input/segmentationmodelspytorch')
import segmentation_models_pytorch as smp
from segmentation_models_pytorch import Unet, FPN

In [ ]:
if CFG.base_model =='Unet':
    base_model = smp.Unet(CFG.encoder, encoder_weights='imagenet', classes=1)
if CFG.base_model =='FPN':
    base_model = smp.FPN(CFG.encoder, encoder_weights='imagenet', classes=1)
print(base_model)

In [ ]:
class HuBMAP(nn.Module):
    def __init__(self):
        super(HuBMAP, self).__init__()
        self.cnn_model = base_model
        
        #self.cnn_model.decoder.blocks.append(self.cnn_model.decoder.blocks[-1])
        #self.cnn_model.decoder.blocks[-2] = self.cnn_model.decoder.blocks[-3]
    
    def forward(self, imgs):
        img_segs = self.cnn_model(imgs)
        return img_segs

## Loss Function

In [ ]:
sys.path.append('../input/segloss/Segmentationloss/SegLoss-master')
from losses_pytorch.hausdorff import HausdorffDTLoss
from losses_pytorch.lovasz_loss import LovaszSoftmax

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=CFG.smoothing):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return dice
    
    
    
class DiceBCELoss(nn.Module):
    # Formula Given above.
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=CFG.smoothing):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).mean()                            
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.mean() + targets.mean() + smooth)  
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss
        
        return Dice_BCE.mean()
    
    
class Hausdorff_loss(nn.Module):
    def __init__(self):
        super(Hausdorff_loss, self).__init__()
        
    def forward(self, inputs, targets):
        return HausdorffDTLoss()(inputs, targets)
    
    
class Lovasz_loss(nn.Module):
    def __init__(self):
        super(Lovasz_loss, self).__init__()
        
    def forward(self, inputs, targets):
        return LovaszSoftmax()(inputs, targets)
    

In [ ]:
if CFG.criterion == 'DiceBCELoss':
    criterion = DiceBCELoss()
elif CFG.criterion == 'DiceLoss':
    criterion = DiceLoss()
elif CFG.criterion == 'Hausdorff':
    criterion = Hausdorff_loss()
elif CFG.criterion == 'Lovasz':
    criterion = Lovasz_loss()

## Train Function

In [ ]:
def HuBMAPLoss(images, targets, model, device, loss_func=criterion):
    model.to(device)
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    loss_func = loss_func
    loss = loss_func(outputs, targets)
    return loss, outputs

In [ ]:
def train_one_epoch(epoch, model, device, optimizer, scheduler, trainloader):
    model.train()
    t = time.time()
    total_loss = 0
    for step, (images, targets) in enumerate(trainloader):
        loss, outputs = HuBMAPLoss(images, targets, model, device)
        loss.backward()
        if ((step+1)%4==0 or (step+1)==len(trainloader)):
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        loss = loss.detach().item()
        total_loss += loss
        if ((step+1)%10==0 or (step+1)==len(trainloader)):
            print(
                    f'epoch {epoch} train step {step+1}/{len(trainloader)}, ' + \
                    f'loss: {total_loss/len(trainloader):.4f}, ' + \
                    f'time: {(time.time() - t):.4f}', end= '\r' if (step + 1) != len(trainloader) else '\n'
                )

            
        
def valid_one_epoch(epoch, model, device, optimizer, scheduler, validloader):
    model.eval()
    t = time.time()
    total_loss = 0
    for step, (images, targets) in enumerate(validloader):
        loss, outputs = HuBMAPLoss(images, targets, model, device)
        loss = loss.detach().item()
        total_loss += loss
        if ((step+1)%4==0 or (step+1)==len(validloader)):
            scheduler.step(total_loss/len(validloader))
        if ((step+1)%10==0 or (step+1)==len(validloader)):
            print(
                    f'epoch {epoch} trainz step {step+1}/{len(validloader)}, ' + \
                    f'loss: {total_loss/len(validloader):.4f}, ' + \
                    f'time: {(time.time() - t):.4f}', end= '\r' if (step + 1) != len(validloader) else '\n'
                )

## Creating Folds Column

In [ ]:
FOLDS = 5
gkf = GroupKFold(FOLDS)
dir_df['Folds'] = 0
for fold, (tr_idx, val_idx) in enumerate(gkf.split(dir_df, groups=dir_df[dir_df.columns[0]].values)):
    dir_df.loc[val_idx, 'Folds'] = fold

In [ ]:
dir_df

## The Real Training

In [ ]:
def prepare_train_valid_dataloader(df, fold):
    train_ids = df[~df.Folds.isin(fold)]
    val_ids = df[df.Folds.isin(fold)]
    
    train_ds = HuBMAPDataset(train_ids, train='train', augment='base', transform=True)
    val_ds = HuBMAPDataset(val_ids, train='val', augment='base', transform=True)
    train_loader = DataLoader(train_ds, batch_size=4, pin_memory=True, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_ds, batch_size=4, pin_memory=True, shuffle=False, num_workers=4)
    return train_loader, val_loader

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = HuBMAP().to(device)
#optimizer
optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, amsgrad=False)

# scheduler setting
if CFG.scheduler == 'CosineAnnealingWarmRestarts':
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=1, eta_min=CFG.min_lr, last_epoch=-1)
elif CFG.scheduler == 'ReduceLROnPlateau':
    scheduler = ReduceLROnPlateauReduceLROnPlateau(optimizer, mode='min', factor=CFG.factor, patience=CFG.patience, verbose=True, eps=CFG.eps)
elif CFG.scheduler == 'CosineAnnealingLR':
    scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)

In [ ]:
for fold, (tr_idx, val_idx) in enumerate(gkf.split(dir_df, groups=dir_df[dir_df.columns[0]].values)):
    if fold>1:
        break
    trainloader, validloader = prepare_train_valid_dataloader(dir_df, [fold])

    num_epochs = CFG.epoch
    #num_epochs = 2
    for epoch in range(num_epochs):
        train_one_epoch(epoch, model, device, optimizer, scheduler, trainloader)
        with torch.no_grad():
            valid_one_epoch(epoch, model, device, optimizer, scheduler, validloader)
    torch.save(model.state_dict(),f'FOLD-{fold}-model.pth')